In [12]:
import gffutils
import pandas as pd
import os
os.chdir("/home/tfuruta1/CHM13_changes")

In [13]:
original = gffutils.FeatureDB("data/original.db")
mapped = gffutils.FeatureDB("data/mapped.db")

In [14]:

remove = pd.read_csv("changes/1_synonyms_remove.csv", index_col = 0)
add = pd.read_csv("changes/1_synonyms_add.csv", index_col = 0)

In [15]:
remove

,gene,chromosome,start,end,strand,biotype
1,KIAA1522,chr1,32601643,32634754,+,protein_coding
2,FAM183A,chr1,43013445,43026887,+,protein_coding
3,LEXM,chr1,54686281,54722525,+,protein_coding
6,FAM102B,chr1,108593088,108672314,+,protein_coding
7,PIFO,chr1,111339352,111367700,+,protein_coding
...,...,...,...,...,...,...
414,LOC102724428,chr21,41769738,41782385,-,protein_coding
423,LOC112268293,chr22,40431389,40435898,+,protein_coding
435,LOC122394733,chrX,102403657,102407006,+,protein_coding
437,LOC727838,chrX,117450728,117454503,+,protein_coding


In [16]:
add

,gene,chromosome,start,end,strand,biotype
1,NHSL3,chr1,32601643,32634754,+,protein_coding
2,CFAP144,chr1,43013445,43026887,+,protein_coding
3,CIMAP2,chr1,54686281,54722525,+,protein_coding
6,EEIG2,chr1,108593088,108672314,+,protein_coding
7,CIMAP3,chr1,111339352,111367700,+,protein_coding
...,...,...,...,...,...,...
414,SIK1,chr21,41769738,41782385,-,protein_coding
423,TMA7B,chr22,40431389,40435898,+,protein_coding
435,TMSB15C,chrX,102403657,102407006,+,protein_coding
437,CT47C1,chrX,117450728,117454503,+,protein_coding


In [ ]:
for gene in original.features_of_type("gene"):
    
    gene_name = (gene.attributes.get("gene") or gene.attributes.get("gene_name"))
    if not gene_name:
        continue
    gene_name = gene_name[0]
    match = remove[
        (remove["gene"] == gene_name)
        & (remove['chromosome'] == gene.seqid)
        & (remove['start'] == gene.start)
        & (remove['end'] == gene.end)
        & (remove['strand'] == gene.strand)
    ]
    if not match.empty:
        original.delete(gene, children = True)
        print(f"Deleted {gene}")

Deleted chr1	Liftoff	gene	32601643	32634754	.	+	.	ID=KIAA1522;gene_name=KIAA1522;db_xref=HGNC:HGNC:29301;description=KIAA1522;gbkey=Gene;gene=KIAA1522;gene_biotype=protein_coding;coverage=1.0;sequence_ID=0.999;valid_ORFs=4;extra_copy_number=0;copy_num_ID=KIAA1522_0
Deleted chr1	Liftoff	gene	37546290	37556969	.	-	.	ID=C1orf109;gene_name=C1orf109;db_xref=MIM:614799;description=chromosome 1 open reading frame 109;gbkey=Gene;gene=C1orf109;gene_biotype=protein_coding;coverage=1.0;sequence_ID=0.997;valid_ORFs=25;extra_copy_number=0;copy_num_ID=C1orf109_0
Deleted chr1	Liftoff	gene	43013445	43026887	.	+	.	ID=FAM183A;gene_name=FAM183A;db_xref=HGNC:HGNC:34347;description=family with sequence similarity 183 member A;gbkey=Gene;gene=FAM183A;gene_biotype=protein_coding;coverage=1.0;sequence_ID=0.996;valid_ORFs=3;extra_copy_number=0;copy_num_ID=FAM183A_0
Deleted chr1	Liftoff	gene	53794486	53829213	.	-	.	ID=HSPB11;gene_name=HSPB11;db_xref=HGNC:HGNC:25019;description=heat shock protein family B (small

In [26]:
gene_name

'DDX11L16'

In [9]:
genes_added_num = 0
for _, row in add.iterrows():
    gene_name = row['gene']
    chrom = row['chromosome']
    start = row['start']
    end = row['end']
    strand = row['strand']
    for gene in mapped.features_of_type('gene'):
        gname = (
            gene.attributes.get("gene") or
            gene.attributes.get("gene_name")
        )
        if not gene_name:
            continue
        gene_name = gene_name[0]
        if (
            (gname == gene_name)
            & (gene.seqid == chrom)
            & (gene.start == start)
            & (gene.end == end)
            & (gene.strand == strand)
        ):
            original.update([gene])
            print(gene)
            gene_name += 1
            for descendants in mapped.children(gene, level=None, order_by='start'):
                original.update([descendants])
        

In [10]:
with open('data/new_annotation.gff', 'w') as out:
    for feature in original.all_features(order_by=('seqid', 'start')):
        out.write(str(feature) + '\n')